In [122]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import keras

everything = keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

cast_x_train = tf.cast(x_train, 'float32')
cast_y_train = tf.cast(y_train, 'float32')
cast_x_test = tf.cast(x_test, 'float32')
cast_y_test = tf.cast(y_test, 'float32')
#print(everything)

In [123]:
epochs = 10
classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128

ein neurales netzwerk - spiegeln?

In [124]:
def recurrent_neural_network(data):
    layer = {'weights':tf.Variable(tf.random.normal([rnn_size, classes])),
            'biases':tf.Variable(tf.random.normal([classes]))}

    data = tf.transpose(data, [1, 0, 2])
    data = tf.reshape(data, [-1, chunk_size])
    data = tf.split(data, n_chunks, 0)

    lstm_cell = rnn_cell.BasicLSTMCell(rnn_size, state_is_tuple=True)
    outputs, states = rnn.static_rnn(lstm_cell, data, dtype=tf.float32)

    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']

    return output

In [125]:
def train_neural_network(data):
    prediction = recurrent_neural_network(data)
    
    print(type(prediction))
    print(prediction.shape)
    print(cast_y_train.shape)

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=cast_y_train))
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    with tf.Session() as session:
        session.run(tf.initialize_all_variables())

        for epoch in range(int(x_train.num_examples/batch_size)):
            epoch_loss = 0
            epoch_x, epoch_y = x_train.next_batch(batch_size)
            epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))

            _, c = session.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
            epoch_loss += c

        print('Epoch', epoch, 'completed oput of', epochs, 'loss:', epoch_loss)

    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy:', accuracy.eval({x_test.images.reshape((-1, n_chunks, chunk_size)), y_test.labels}))

In [130]:
train_neural_network(cast_x_train)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(60000, 10)
(60000,)


InvalidArgumentError: logits and labels must be broadcastable: logits_size=[60000,10] labels_size=[1,60000] [Op:SoftmaxCrossEntropyWithLogits]

In [126]:
#test = tf.convert_to_tensor(x_train, dtype='float32')
#test2 = tf.cast(test, 'float32')

cast_x_train = tf.cast(x_train, 'float32')
cast_y_train = tf.cast(y_train, 'float32')
cast_x_test = tf.cast(x_test, 'float32')
cast_y_test = tf.cast(y_test, 'float32')




#cast doesnt change shape
print(x_train.shape)
print(cast_x_train.shape)

print(y_train.shape)
print(cast_y_train.shape)

(60000, 28, 28)
(60000, 28, 28)
(60000,)
(60000,)


In [127]:
train_neural_network(tf.cast(x_train, 'float32'))

<class 'tensorflow.python.framework.ops.EagerTensor'>
(60000, 10)
(60000,)


InvalidArgumentError: logits and labels must be broadcastable: logits_size=[60000,10] labels_size=[1,60000] [Op:SoftmaxCrossEntropyWithLogits]